# PurpleAir PM & Temperature Interpolation

## Set Up

### Import Packages

In [1]:
import arcpy
import pandas as pd
import psycopg2
from psycopg2 import sql
import numpy as np
import os
import folium

### Set Workspace

In [2]:
cwd = os.getcwd()

# Make it workspace

arcpy.env.workspace = os.path.join(cwd, '..', '..', 'data', 'QAQC.gdb')

arcpy.env.overwriteOutput = True # Overwrite layers is okay

save_path_csv = os.path.join(cwd, '..', '..', 'data')
save_path_gdb = os.path.join(cwd, '..', '..', 'data', 'QAQC.gdb')
historic_path = os.path.join(cwd, '..', '..', 'data', 'purpleair_historic.csv')
station_path = os.path.join(cwd, '..', '..', 'data', 'purpleair_stations.csv')
boundary_path = os.path.join(cwd, '..', '..', 'data', 'mpls_boundary.geojson')

In [3]:
output_fc = 'mpls_boundary'
output_fc_path = os.path.join(save_path_gdb, output_fc)

# Convert the GeoJSON file to a feature class
arcpy.JSONToFeatures_conversion(boundary_path, output_fc)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\mpls_boundary'>

### Selecting a Single Day to Interpolate

In [4]:
df = pd.read_csv(historic_path)

# Convert the 'timestamp' column to a datetime object
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Filter the rows that have a timestamp of 4/1/23
date_filter = df['timestamp'].dt.date == pd.to_datetime('4/1/23').date()
filtered_df = df[date_filter]

filtered_df.to_csv('historic_april1.csv', index=False)
historic_filtered_path = os.path.join(cwd, '..', '..', 'data', 'historic_april1.csv')

### Merge CSVs to get Lat/Longs with PM Data

In [5]:
# Load the first CSV file into a DataFrame
df1 = pd.read_csv(station_path)

# Load the second CSV file into a DataFrame
df2 = pd.read_csv(historic_filtered_path)

# Merge the two DataFrames together on a common column
merged_df = pd.merge(df1, df2, on='sensor_index')

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('merged_file.csv', index=False)
merged_path = os.path.join(cwd, '..', '..', 'data', 'merged_file.csv')

In [6]:
# Set output point feature class name and path
output_point_fc = "stations_XYTableToPoint"
output_point_path = os.path.join(save_path_gdb, output_point_fc)

# Specify x,y fields
x_field = "longitude"
y_field = "latitude"

# Use XYTableToPoint tool to create point features
station_points = arcpy.management.XYTableToPoint(merged_path, output_point_path, x_field, y_field)

In [7]:
#This was my attempt instead of doing the CSV merge, but was unsuccesful
'''
output_table = "joined_table"
arcpy.management.AddJoin(
    in_layer_or_view=station_points,
    in_field="sensor_index",
    join_table=historic_filtered_path,
    join_field="sensor_index",
    join_type="KEEP_ALL",
    index_join_fields="NO_INDEX_JOIN_FIELDS"
)

# Export the joined layer to a new table
historic_apr1 = arcpy.management.CopyRows("stations_XYTableToPoint", output_table)
'''

'\noutput_table = "joined_table"\narcpy.management.AddJoin(\n    in_layer_or_view=station_points,\n    in_field="sensor_index",\n    join_table=historic_filtered_path,\n    join_field="sensor_index",\n    join_type="KEEP_ALL",\n    index_join_fields="NO_INDEX_JOIN_FIELDS"\n)\n\n# Export the joined layer to a new table\nhistoric_apr1 = arcpy.management.CopyRows("stations_XYTableToPoint", output_table)\n'

## PM 2.5 Interpolation

### Kriging PM 2.5

In [8]:
out_kriging_raster = "Kriging_Apr1"
output_raster_path = os.path.join(save_path_gdb, out_kriging_raster)
arcpy.ga.EmpiricalBayesianKriging(
    in_features="stations_XYTableToPoint",
    z_field="pm2_5",
    out_ga_layer="pm2_5_kriging_stats",
    out_raster=out_kriging_raster,
    cell_size=0.000932919999999967,
    transformation_type="NONE",
    max_local_points=100,
    overlap_factor=1,
    number_semivariograms=100,
    search_neighborhood="NBRTYPE=StandardCircular RADIUS=9.87111293434023E-02 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    output_type="PREDICTION",
    quantile_value=0.5,
    threshold_type="EXCEED",
    probability_threshold=None,
    semivariogram_model_type="POWER"
)
output_raster_path = os.path.join(save_path_gdb, out_kriging_raster)

### IDW PM 2.5

In [9]:
out_idw_raster = "IDW_Apr1"
arcpy.ga.IDW(
    in_features="stations_XYTableToPoint",
    z_field="pm2_5",
    out_ga_layer="pm2_5_idw_stats",
    out_raster=out_idw_raster,
    cell_size=10,
    power=2,
    search_neighborhood="NBRTYPE=Standard S_MAJOR=9.87111293434023E-02 S_MINOR=9.87111293434023E-02 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    weight_field=None
)

<Result 'pm2_5_idw_stats'>

### Local Polynomial PM 2.5

In [10]:
out_poly_raster = "Poly_Apr1"
arcpy.ga.LocalPolynomialInterpolation(
    in_features="stations_XYTableToPoint",
    z_field="pm2_5",
    out_ga_layer="pm2_5_poly_stats",
    out_raster=out_poly_raster,
    cell_size=0.000932919999999967,
    power=1,
    search_neighborhood="NBRTYPE=Standard S_MAJOR=9.87111293434023E-02 S_MINOR=9.87111293434023E-02 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    kernel_function="EXPONENTIAL",
    bandwidth=None,
    use_condition_number="NO_USE_CONDITION_NUMBER",
    condition_number=None,
    weight_field=None,
    output_type="PREDICTION"
)

<Result 'pm2_5_poly_stats'>

## Temperature Interpolation

### Kriging Temperature

In [11]:
out_kriging_temp_raster = "Kriging_Temp_Apr1"
arcpy.ga.EmpiricalBayesianKriging(
    in_features="stations_XYTableToPoint",
    z_field="temperature",
    out_ga_layer="temp_kriging_stats",
    out_raster=out_kriging_temp_raster,
    cell_size=0.000932919999999967,
    transformation_type="NONE",
    max_local_points=100,
    overlap_factor=1,
    number_semivariograms=100,
    search_neighborhood="NBRTYPE=StandardCircular RADIUS=9.87111293434023E-02 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    output_type="PREDICTION",
    quantile_value=0.5,
    threshold_type="EXCEED",
    probability_threshold=None,
    semivariogram_model_type="POWER"
)

<Result 'temp_kriging_stats'>

### IDW Temp

In [12]:
out_idw_temp_raster = "IDW_Temp_Apr1"
arcpy.ga.IDW(
    in_features="stations_XYTableToPoint",
    z_field="temperature",
    out_ga_layer="temp_idw_stats",
    out_raster=out_idw_temp_raster,
    cell_size=0.000932919999999967,
    power=2,
    search_neighborhood="NBRTYPE=Standard S_MAJOR=9.87111293434023E-02 S_MINOR=9.87111293434023E-02 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    weight_field=None
)

<Result 'temp_idw_stats'>

### Local Polynomial Temp

In [13]:
out_poly_temp_raster = "Poly_Temp_Apr1"
arcpy.ga.LocalPolynomialInterpolation(
    in_features="stations_XYTableToPoint",
    z_field="temperature",
    out_ga_layer="temp_poly_stats",
    out_raster=out_poly_temp_raster,
    cell_size=0.000932919999999967,
    power=1,
    search_neighborhood="NBRTYPE=Standard S_MAJOR=9.87111293434023E-02 S_MINOR=9.87111293434023E-02 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    kernel_function="EXPONENTIAL",
    bandwidth=None,
    use_condition_number="NO_USE_CONDITION_NUMBER",
    condition_number=None,
    weight_field=None,
    output_type="PREDICTION"
)

<Result 'temp_poly_stats'>

## Cross Validation

### Kriging PM 2.5

In [14]:
#cross validation
pm2_5_kriging_fc = "pm2_5_kriging_cross_valid"
pm2_5_kriging_path = os.path.join(save_path_gdb, pm2_5_kriging_fc)
arcpy.ga.CrossValidation(
    in_geostat_layer="pm2_5_kriging_stats",
    out_point_feature_class=pm2_5_kriging_fc
)
# add errors to shared PD DF
input_fc ="pm2_5_kriging_cross_valid"
fields = ["OBJECTID", "Error"]

# Convert the feature class to a NumPy array and create a Pandas DataFrame
arr = arcpy.da.TableToNumPyArray(input_fc, fields)
df = pd.DataFrame(arr)
rmse_df = df.rename(columns={'Error': 'PM2_5_Kriging_Error'})

### IDW PM 2.5

In [15]:
#cross validation
pm2_5_idw_fc = 'pm2_5_idw_cross_valid'
pm2_5_idw_path = os.path.join(save_path_gdb, pm2_5_idw_fc)
arcpy.ga.CrossValidation(
    in_geostat_layer="pm2_5_idw_stats",
    out_point_feature_class=pm2_5_idw_fc
)
# add errors to shared PD DF
input_fc ='pm2_5_idw_cross_valid'
fields = ["OBJECTID", "Error"]

# Convert the feature class to a NumPy array and create a Pandas DataFrame
arr = arcpy.da.TableToNumPyArray(input_fc, fields)
df = pd.DataFrame(arr)
merged_df = rmse_df.merge(df, on='OBJECTID', how='left')
rmse_df = merged_df.rename(columns={'Error': 'PM2_5_IDW_Error'})

### Local Polynomial PM 2.5

In [16]:
#cross validation
pm2_5_poly_fc = 'pm2_5_poly_cross_valid'
pm2_5_poly_path = os.path.join(save_path_gdb, pm2_5_poly_fc)
arcpy.ga.CrossValidation(
    in_geostat_layer="pm2_5_poly_stats",
    out_point_feature_class=pm2_5_poly_fc
)
# add errors to shared PD DF
input_fc ='pm2_5_poly_cross_valid'
fields = ["OBJECTID", "Error"]

# Convert the feature class to a NumPy array and create a Pandas DataFrame
arr = arcpy.da.TableToNumPyArray(input_fc, fields)
df = pd.DataFrame(arr)
merged_df = rmse_df.merge(df, on='OBJECTID', how='left')
rmse_df = merged_df.rename(columns={'Error': 'PM2_5_Poly_Error'})

### Kriging Temperature

In [17]:
#cross validation
temp_kriging_fc = 'temp_kriging_cross_valid'
temp_kriging_path = os.path.join(save_path_gdb, temp_kriging_fc)
arcpy.ga.CrossValidation(
    in_geostat_layer="temp_kriging_stats",
    out_point_feature_class=temp_kriging_fc
)
# add errors to shared PD DF
input_fc ='temp_kriging_cross_valid'
fields = ["OBJECTID", "Error"]

# Convert the feature class to a NumPy array and create a Pandas DataFrame
arr = arcpy.da.TableToNumPyArray(input_fc, fields)
df = pd.DataFrame(arr)
merged_df = rmse_df.merge(df, on='OBJECTID', how='left')
rmse_df = merged_df.rename(columns={'Error': 'Temp_Kriging_Error'})

### IDW Temprature

In [18]:
#cross validation
temp_idw_fc = 'temp_idw_cross_valid'
temp_idw_path = os.path.join(save_path_gdb, temp_idw_fc)
arcpy.ga.CrossValidation(
    in_geostat_layer="temp_idw_stats",
    out_point_feature_class=temp_idw_fc
)
# add errors to shared PD DF
input_fc ='temp_idw_cross_valid'
fields = ["OBJECTID", "Error"]

# Convert the feature class to a NumPy array and create a Pandas DataFrame
arr = arcpy.da.TableToNumPyArray(input_fc, fields)
df = pd.DataFrame(arr)
merged_df = rmse_df.merge(df, on='OBJECTID', how='left')
rmse_df = merged_df.rename(columns={'Error': 'Temp_IDW_Error'})

### Local Polynomial Temperature

In [19]:
#cross validation
temp_poly_fc = 'temp_poly_cross_valid'
temp_poly_path = os.path.join(save_path_gdb, temp_poly_fc)
arcpy.ga.CrossValidation(
    in_geostat_layer="temp_poly_stats",
    out_point_feature_class=temp_poly_fc
)
# add errors to shared PD DF
input_fc ='temp_poly_cross_valid'
fields = ["OBJECTID", "Error"]

# Convert the feature class to a NumPy array and create a Pandas DataFrame
arr = arcpy.da.TableToNumPyArray(input_fc, fields)
df = pd.DataFrame(arr)
merged_df = rmse_df.merge(df, on='OBJECTID', how='left')
rmse_df = merged_df.rename(columns={'Error': 'Temp_Poly_Error'})

In [20]:
rmse_df

,OBJECTID,PM2_5_Kriging_Error,PM2_5_IDW_Error,PM2_5_Poly_Error,Temp_Kriging_Error,Temp_IDW_Error,Temp_Poly_Error
0,1,1.886304,2.092020,1.909499,-3.303338,-3.470032,-3.575083
1,2,3.034219,3.066576,1.987676,-0.591053,-2.571424,-1.839374
2,3,-26.045040,-27.152979,-26.524731,9.709327,9.465443,9.367046
3,4,-0.686374,-1.163698,-0.944816,-4.381915,-5.183978,-1.712790
4,5,0.576673,-0.204923,0.335255,-2.131327,-0.291158,-0.087575
...,...,...,...,...,...,...,...
72,73,1.762010,0.690085,-2.485621,1.424139,1.873297,2.286915
73,74,3.239165,2.953680,3.080456,-4.242841,-1.259224,-1.153477
74,75,3.492016,3.110879,2.813213,10.605938,-4.826753,-35.120113
75,76,2.384410,2.396766,2.005166,NaN,NaN,NaN


## Calculate RMSE

In [21]:
#calculate RMSE for each cross valid error
rmse_df['SQ_PM2_5_Kriging_Error'] = rmse_df['PM2_5_Kriging_Error'] ** 2
pm2_5_kriging_rmse = numpy.sqrt(rmse_df['SQ_PM2_5_Kriging_Error'].mean())

In [22]:
rmse_df['SQ_PM2_5_IDW_Error'] = rmse_df['PM2_5_IDW_Error'] ** 2
pm2_5_idw_rmse = numpy.sqrt((rmse_df['SQ_PM2_5_IDW_Error'].values).mean())

In [23]:
rmse_df['SQ_PM2_5_Poly_Error'] = rmse_df['PM2_5_Poly_Error'] ** 2
pm2_5_poly_rmse = numpy.sqrt(rmse_df['SQ_PM2_5_Poly_Error'].mean())

In [24]:
# compare all of the RMSEs to find the best
pm2_5_compare = pd.DataFrame({'method':['kriging', 'idw', 'local polynomial'],
                            'RMSE':[pm2_5_kriging_rmse, pm2_5_idw_rmse, pm2_5_poly_rmse]})
# Get index of row with lowest value of rmse
min_index = pm2_5_compare['RMSE'].idxmin()

# Get method with lowest rmse
min_id = pm2_5_compare.loc[min_index, 'method']
print("Method with the lowest PM2.5 RMSE value is", min_id)

Method with the lowest PM2.5 RMSE value is kriging


In [25]:
rmse_df['SQ_Temp_Kriging_Error'] = rmse_df['Temp_Kriging_Error'] ** 2
temp_kriging_rmse = numpy.sqrt(rmse_df['SQ_Temp_Kriging_Error'].mean())

In [26]:
rmse_df['SQ_Temp_IDW_Error'] = rmse_df['Temp_IDW_Error'] ** 2
temp_idw_rmse = numpy.sqrt((rmse_df['SQ_Temp_IDW_Error'].values).mean())

In [27]:
rmse_df['SQ_Temp_Poly_Error'] = rmse_df['Temp_Poly_Error'] ** 2
temp_poly_rmse = numpy.sqrt(rmse_df['SQ_Temp_Poly_Error'].mean())

In [28]:
temp_compare = pd.DataFrame({'method':['kriging', 'idw', 'local polynomial'],
                            'RMSE':[temp_kriging_rmse, temp_idw_rmse, temp_poly_rmse]})
# Get index of row with lowest value of rmse
min_index = temp_compare['RMSE'].idxmin()

# Get method with lowest rmse
min_id = temp_compare.loc[min_index, 'method']
print("Method with the lowest Temperature RMSE value is", min_id)

Method with the lowest Temperature RMSE value is kriging


## Raster to Point Each Interpolation

### Kriging PM 2.5

In [29]:
out_point_kriging_pm = "Kriging_PM2_5_Raster_to_Point"
output_raster_path = os.path.join(save_path_gdb, out_point_kriging_pm)
arcpy.conversion.RasterToPoint(
    in_raster="Kriging_Apr1",
    out_point_features=out_point_kriging_pm,
    raster_field="Value"
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\Kriging_PM2_5_Raster_to_Point'>

### IDW PM 2.5

In [30]:
out_point_idw_pm = "IDW_PM2_5_Raster_to_Point"
output_raster_path = os.path.join(save_path_gdb, out_point_idw_pm)
arcpy.conversion.RasterToPoint(
    in_raster="IDW_Apr1",
    out_point_features=out_point_idw_pm,
    raster_field="Value"
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\IDW_PM2_5_Raster_to_Point'>

### Local Polynomial PM 2.5

In [31]:
out_point_poly_pm = "Poly_PM2_5_Raster_to_Point"
output_raster_path = os.path.join(save_path_gdb, out_point_poly_pm)
arcpy.conversion.RasterToPoint(
    in_raster="Poly_Apr1",
    out_point_features=out_point_poly_pm,
    raster_field="Value"
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\Poly_PM2_5_Raster_to_Point'>

### Kriging Temp

In [32]:
out_point_kriging_temp = "Kriging_Temp_Raster_to_Point"
output_raster_path = os.path.join(save_path_gdb, out_point_kriging_temp)
arcpy.conversion.RasterToPoint(
    in_raster="Kriging_Temp_Apr1",
    out_point_features=out_point_kriging_temp,
    raster_field="Value"
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\Kriging_Temp_Raster_to_Point'>

### IDW Temp

In [33]:
out_point_idw_temp = "IDW_Temp_Raster_to_Point"
output_raster_path = os.path.join(save_path_gdb, out_point_idw_temp)
arcpy.conversion.RasterToPoint(
    in_raster="IDW_Temp_Apr1",
    out_point_features=out_point_idw_temp,
    raster_field="Value"
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\IDW_Temp_Raster_to_Point'>

### Local Polynomial Temp

In [34]:
out_point_poly_temp = "Poly_Temp_Raster_to_Point"
output_raster_path = os.path.join(save_path_gdb, out_point_poly_temp)
arcpy.conversion.RasterToPoint(
    in_raster="Poly_Temp_Apr1",
    out_point_features=out_point_poly_temp,
    raster_field="Value"
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\Poly_Temp_Raster_to_Point'>

## Clipping Each Interpolation

### Kriging PM 2.5 Clip

In [35]:
out_clip_kriging_pm = "PM2_5_Kriging"
output_clip_path = os.path.join(save_path_gdb, out_clip_kriging_pm)
arcpy.analysis.Clip(
    in_features="Kriging_PM2_5_Raster_to_Point",
    clip_features='mpls_boundary',
    out_feature_class=out_clip_kriging_pm,
    cluster_tolerance=None
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\PM2_5_Kriging'>

### IDW PM 2.5 Clip

In [36]:
out_clip_idw_pm = "PM2_5_IDW"
output_clip_path = os.path.join(save_path_gdb, out_clip_idw_pm)
arcpy.analysis.Clip(
    in_features="IDW_PM2_5_Raster_to_Point",
    clip_features='mpls_boundary',
    out_feature_class=out_clip_idw_pm,
    cluster_tolerance=None
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\PM2_5_IDW'>

### Local Polynomial PM 2.5 Clip

In [37]:
out_clip_poly_pm = "PM2_5_Poly"
output_clip_path = os.path.join(save_path_gdb, out_clip_poly_pm)
arcpy.analysis.Clip(
    in_features="Poly_PM2_5_Raster_to_Point",
    clip_features='mpls_boundary',
    out_feature_class=out_clip_poly_pm,
    cluster_tolerance=None
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\PM2_5_Poly'>

### Kriging Temperature Clip

In [38]:
out_clip_kriging_temp = "Temp_Kriging"
output_clip_path = os.path.join(save_path_gdb, out_clip_kriging_temp)
arcpy.analysis.Clip(
    in_features="Kriging_Temp_Raster_to_Point",
    clip_features='mpls_boundary',
    out_feature_class=out_clip_kriging_temp,
    cluster_tolerance=None
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\Temp_Kriging'>

### IDW Temperature Clip

In [39]:
out_clip_idw_temp = "Temp_IDW"
output_clip_path = os.path.join(save_path_gdb, out_clip_idw_temp)
arcpy.analysis.Clip(
    in_features="IDW_Temp_Raster_to_Point",
    clip_features='mpls_boundary',
    out_feature_class=out_clip_idw_temp,
    cluster_tolerance=None
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\Temp_IDW'>

### Local Polynomial Clip

In [40]:
out_clip_poly_temp = "Temp_Poly"
output_clip_path = os.path.join(save_path_gdb, out_clip_poly_temp)
arcpy.analysis.Clip(
    in_features="Poly_Temp_Raster_to_Point",
    clip_features='mpls_boundary',
    out_feature_class=out_clip_poly_temp,
    cluster_tolerance=None
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\Temp_Poly'>

## Save to Local and Remote Databases

### Connect to DB

In [41]:
# Get credentials

cred_pth = os.path.join(os.getcwd(), '..', '..', 'database', 'db_credentials.txt')

with open(cred_pth, 'r') as f:
    
    creds = f.readlines()[0].split(', ')

# Connect to PostGIS Database

pg_connection_dict = dict(zip(['dbname', 'user', 'password', 'port', 'host'], creds))

try:
    conn = psycopg2.connect(**pg_connection_dict)
    print("connected")
except:
    print("connection failed")

connected


### Create & Populate Table

In [42]:
points = os.path.join(save_path_gdb, "PM2_5_Kriging")
fields_points = ['pointid', 'grid_code', "SHAPE@WKT"]

# Create SQL table
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS PM2_5_Kriging")
cursor.execute("""
    CREATE TABLE PM2_5_Kriging (
        id SERIAL,
        pointid INT,
        grid_code DOUBLE PRECISION,
        geometry geometry
        )
""")
conn.commit()

# Populate PostGIS
with arcpy.da.SearchCursor(points, fields_points) as da_cursor:
    for row in da_cursor:
        wkt = row[2]
        cursor.execute("INSERT INTO PM2_5_Kriging (pointid, grid_code, geometry) VALUES (%s, %s, ST_GeomFromText(%s, 4326))", (row[0], row[1], wkt))
        conn.commit()

In [43]:
points = os.path.join(save_path_gdb, "Temp_Kriging")
fields_points = ['pointid', 'grid_code', "SHAPE@WKT"]

# Create SQL table
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS Temp_Kriging")
cursor.execute("""
    CREATE TABLE Temp_Kriging (
        id SERIAL,
        pointid INT,
        grid_code DOUBLE PRECISION,
        geometry geometry
        )
""")
conn.commit()

# Populate PostGIS
with arcpy.da.SearchCursor(points, fields_points) as da_cursor:
    for row in da_cursor:
        wkt = row[2]
        cursor.execute("INSERT INTO Temp_Kriging (pointid, grid_code, geometry) VALUES (%s, %s, ST_GeomFromText(%s, 4326))", (row[0], row[1], wkt))
        conn.commit()

In [44]:
points = os.path.join(save_path_gdb, "pm2_5_kriging_cross_valid")
fields_points = ['OBJECTID','Error', "SHAPE@WKT"]

# Create SQL table
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS PM2_5_Kriging_Errors")
cursor.execute("""
    CREATE TABLE PM2_5_Kriging_Errors (
        id SERIAL,
        objectid INT,
        error DOUBLE PRECISION,
        geometry geometry
        )
""")
conn.commit()

# Populate PostGIS
with arcpy.da.SearchCursor(points, fields_points) as da_cursor:
    for row in da_cursor:
        wkt = row[2]
        cursor.execute("INSERT INTO PM2_5_Kriging_Errors (objectid, error, geometry) VALUES (%s, %s, ST_GeomFromText(%s, 4326))", (row[0], row[1], wkt))
        conn.commit()

In [45]:
points = os.path.join(save_path_gdb, "temp_kriging_cross_valid")
fields_points = ['OBJECTID','Error', "SHAPE@WKT"]

# Create SQL table
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS Temp_Kriging_Errors")
cursor.execute("""
    CREATE TABLE Temp_Kriging_Errors (
        id SERIAL,
        objectid INT,
        error DOUBLE PRECISION,
        geometry geometry
        )
""")
conn.commit()

# Populate PostGIS
with arcpy.da.SearchCursor(points, fields_points) as da_cursor:
    for row in da_cursor:
        wkt = row[2]
        cursor.execute("INSERT INTO Temp_Kriging_Errors (objectid, error, geometry) VALUES (%s, %s, ST_GeomFromText(%s, 4326))", (row[0], row[1], wkt))
        conn.commit()